#### Example queries from Multi-omic provider.

##### This jupyter notebook give an example to query data from mutation dependent drug response 

In [8]:
import requests

import pandas as pd
def Query_fields():
    query_str = "https://biothings.ncats.io/drug_response_kp/metadata/fields"
    response = requests.get(query_str)
    result = response.json()
    return(result)

def format_result(result):
    subject_id = []
    subject_symbol = []
    subject_type = []

    object_id = []
    object_name = []
    object_type = []

    edge_context = []
    edge_label = []
    edge_effect_size = []
    edge_ic50s_mut = []
    edge_ic50s_wt = []
    edge_median_ic50_mut = []
    edge_median_ic50_wt = []
    edge_provided_by = []
    edge_prevenence_publications = []
    edge_confidence_p = []
    edge_confidence_sample_size = []
    edge_confidence_mut_size = []
    edge_confidence_wt_size = []
    
    for i in range(0,len(result['hits'])):
        #Edges and context
        edge_context.append(result['hits'][i]['association']['context']['disease']['mondo'])
        edge_label.append(result['hits'][i]['association']['edge_label'])
        edge_effect_size.append(result['hits'][i]['association']['effect_size'])
        edge_ic50s_mut.append(result['hits'][i]['association']['ic50s_mut'])
        edge_ic50s_wt.append(result['hits'][i]['association']['ic50s_wt'])
        edge_median_ic50_mut.append(result['hits'][i]['association']['median_ic50_mut'])
        edge_median_ic50_wt.append(result['hits'][i]['association']['median_ic50_wt'])
        edge_provided_by.append(result['hits'][i]['association']['provided_by'])
        edge_prevenence_publications.append(result['hits'][i]['association']['publications'])
        edge_confidence_p.append(result['hits'][i]['association']['pvalue'])
        edge_confidence_sample_size.append(result['hits'][i]['association']['sample_size'])
        edge_confidence_mut_size.append(result['hits'][i]['association']['size_mut'])
        edge_confidence_wt_size.append(result['hits'][i]['association']['size_wt'])

        #Subject
        if 'NCBIGene' in result['hits'][i]['subject']:
            subject_id.append(result['hits'][i]['subject']['NCBIGene'])
        else:
            subject_id.append("")
            
        subject_symbol.append(result['hits'][i]['subject']['SYMBOL'])
        subject_type.append(result['hits'][i]['subject']['type'])

        #Object
        object_id.append(result['hits'][i]['object']['id'])
        object_name.append(result['hits'][i]['object']['name'])
        object_type.append(result['hits'][i]['object']['type'])

    #Generating dataFrame for the results
    result_df = pd.DataFrame({
    "subject_id":subject_id,
    "subject_symbol": subject_symbol,
    "subject_type": subject_type,
    "object_id":object_id,
    "object_name":object_name,
    "object_type":object_type,
    "edge_label": edge_label,
    "edge_context_disease": edge_context,
    "edge_confidence_p":edge_confidence_p,
    "edge_effect_size":edge_effect_size,
    "edge_median_ic50_wt":edge_median_ic50_wt,
    "edge_median_ic50_mut":edge_median_ic50_mut,
    "edge_confidence_sample_size":edge_confidence_sample_size,
    "edge_confidence_mut_size":edge_confidence_mut_size,
    "edge_confidence_wt_size":edge_confidence_wt_size})
    return(result_df)


def Query_DrugResponse_KP(Query):
    Query_str = "https://biothings.ncats.io/drug_response_kp/query?q="
    
    count = 0
    for i in Query:
        count = count + 1
        if count != len(Query):
            Query_str = Query_str + i + ":"+ Query[i] + "%20AND%20"
        else:
            Query_str = Query_str + i + ":"+ Query[i]+"&size=1000"
    print(Query_str)
    response = requests.get(Query_str)
    result = response.json()
    return(result)

#### Browse the field info in the KG

In [11]:
#Get to know the fields in the KG
result = Query_fields()
result

{'association': {'index': True, 'type': 'object'},
 'association.context': {'index': True, 'type': 'object'},
 'association.context.disease': {'index': True, 'type': 'object'},
 'association.context.disease.id': {'index': True, 'type': 'keyword'},
 'association.context.disease.mondo': {'index': True, 'type': 'keyword'},
 'association.edge_label': {'index': True, 'type': 'keyword'},
 'association.effect_size': {'index': True, 'type': 'float'},
 'association.ic50s_mut': {'index': True, 'type': 'float'},
 'association.ic50s_wt': {'index': True, 'type': 'float'},
 'association.median_ic50_mut': {'index': True, 'type': 'float'},
 'association.median_ic50_wt': {'index': True, 'type': 'float'},
 'association.method': {'index': True, 'type': 'keyword'},
 'association.provided_by': {'index': True, 'type': 'keyword'},
 'association.publications': {'index': True, 'type': 'keyword'},
 'association.pvalue': {'index': True, 'type': 'float'},
 'association.sample_size': {'index': True, 'type': 'integ

####  Set up the query

In [564]:
#https://www.w3schools.com/tags/ref_urlencode.ASP

Query = {"subject.SYMBOL":"EGFR",  #Query by gene symbol
         #"subject.id": "%22NCBIGene:4893%22" #Query by id
        # "association.effect_size": "%3C%2D1", #effect_size, %3C: <; %2D1: -1
         "association.effect_size": "%3C0", #effect_size, %3C: <; %2D1: -1
         "association.pvalue": "%3C0.05", ##p value from t-test 
         #"association.context.disease.mondo": "%22MONDO:0005061%22", #Disease type
         "association.median_ic50_mut":"%3C0", #The median value of IC50 in the mutated group
         #"association.median_ic50_wt":"%3E0"  ##The median value of IC50 in the wild type group
         #"object.name":"Trametinib", #Query by drug
        }

#### Query data from the drug_response_kp

In [565]:
result = Query_DrugResponse_KP(Query)  #Jason format

https://biothings.ncats.io/drug_response_kp/query?q=subject.SYMBOL:EGFR%20AND%20association.effect_size:%3C0%20AND%20association.pvalue:%3C0.05%20AND%20association.median_ic50_mut:%3C0&size=1000


In [569]:
result_df = format_result(result) #Pandas DataFrame format

In [573]:
result_df.sort_values(by = ['edge_confidence_p']) #View the results

,subject_id,subject_symbol,subject_type,object_id,object_name,object_type,edge_label,edge_context_disease,edge_confidence_p,edge_effect_size,edge_median_ic50_wt,edge_median_ic50_mut,edge_confidence_sample_size,edge_confidence_mut_size,edge_confidence_wt_size
3,1956,EGFR,Gene,PUBCHEM:57519523,Afatinib,ChemicalSubstance,related_to,MONDO:0005061,0.000013,-2.180000,1.189939,-2.102841,63,6,46
4,1956,EGFR,Gene,PUBCHEM:123631,Gefitinib,ChemicalSubstance,related_to,MONDO:0005061,0.001952,-1.497063,1.308749,-0.110006,63,6,46
0,1956,EGFR,Gene,PUBCHEM:126941,Methotrexate,ChemicalSubstance,related_to,MONDO:0005170,0.031477,-1.543391,-1.574933,-3.618622,18,3,15
2,1956,EGFR,Gene,PUBCHEM:159324,Tipifarnib,ChemicalSubstance,related_to,"[MONDO:0003093, MONDO:0005580]",0.039579,-1.208813,0.952282,-0.149451,35,4,23
1,1956,EGFR,Gene,CHEMBL.COMPOUND:CHEMBL2107358,Obatoclax Mesylate,ChemicalSubstance,related_to,"[MONDO:0003093, MONDO:0005580]",0.042948,-1.186224,-1.630680,-3.467816,35,4,23
